<a href="https://colab.research.google.com/github/Tinker0425/de-zoomcamp-my-work/blob/master/Homework_2025_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install pyspark
!pip install findspark
!pip install pyngrok

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder \
        .appName('testColab') \
        .getOrCreate()

Manually Stop at ngrok if another spark is running

In [7]:
from pyngrok import ngrok, conf
import getpass

print("Enter your authtoken, which can be copied "
"from https://dashboard.ngrok.com/get-started/your-authtoken")
conf.get_default().auth_token = getpass.getpass()

ui_port = 4040
public_url = ngrok.connect(ui_port).public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{ui_port}\"")

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken
··········


 * ngrok tunnel "https://9e14-34-80-91-86.ngrok-free.app" -> "http://127.0.0.1:4040"


In [25]:
from pyspark import SparkFiles

file_url = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet'
spark.sparkContext.addFile(file_url)

# Read into Spark DF
df = spark.read.parquet(SparkFiles.get('yellow_tripdata_2024-10.parquet'), header=True)

df.count()

3833771

## Question 1

Answer - 3.5.5

In [11]:
spark.version

'3.5.5'

## Question 2

Answer - 25MB

https://www.youtube.com/watch?v=r_Sf6fCB40c&list=PL3MmuxUbc_hJed7dXYoJw8DoCuVHhGEQb&index=47

spark.createDataFrame(df_pandas).schema

In [26]:
# Repartition the Dataframe to 4 partitions and save it to parquet.
df = df.repartition(4)

In [28]:
df.write.parquet('yellow_trip_4_par')

In [30]:
import os
parquet_dir = '/content/yellow_trip_4_par/'
files = os.listdir(parquet_dir)
print(files)

['.part-00001-4b6f94a3-5b5a-40e9-b862-14d9d65cd157-c000.snappy.parquet.crc', 'part-00001-4b6f94a3-5b5a-40e9-b862-14d9d65cd157-c000.snappy.parquet', 'part-00000-4b6f94a3-5b5a-40e9-b862-14d9d65cd157-c000.snappy.parquet', 'part-00002-4b6f94a3-5b5a-40e9-b862-14d9d65cd157-c000.snappy.parquet', '.part-00000-4b6f94a3-5b5a-40e9-b862-14d9d65cd157-c000.snappy.parquet.crc', 'part-00003-4b6f94a3-5b5a-40e9-b862-14d9d65cd157-c000.snappy.parquet', '.part-00002-4b6f94a3-5b5a-40e9-b862-14d9d65cd157-c000.snappy.parquet.crc', '_SUCCESS', '.part-00003-4b6f94a3-5b5a-40e9-b862-14d9d65cd157-c000.snappy.parquet.crc', '._SUCCESS.crc']


In [31]:
# Filter only .parquet files
parquet_files = [f for f in files if f.endswith('.parquet')]
print(parquet_files)
# Corrected version of the last line
parquet_file_sizes = [(f, os.path.getsize(os.path.join(parquet_dir, f))) for f in parquet_files]

# Optionally, convert to MB
parquet_file_sizes_in_MB = [(f, size / (1024 * 1024)) for f, size in parquet_file_sizes]

# Print the result
for file, size in parquet_file_sizes_in_MB:
    print(f"File: {file}, Size: {size:.2f} MB")


['part-00001-4b6f94a3-5b5a-40e9-b862-14d9d65cd157-c000.snappy.parquet', 'part-00000-4b6f94a3-5b5a-40e9-b862-14d9d65cd157-c000.snappy.parquet', 'part-00002-4b6f94a3-5b5a-40e9-b862-14d9d65cd157-c000.snappy.parquet', 'part-00003-4b6f94a3-5b5a-40e9-b862-14d9d65cd157-c000.snappy.parquet']
File: part-00001-4b6f94a3-5b5a-40e9-b862-14d9d65cd157-c000.snappy.parquet, Size: 23.02 MB
File: part-00000-4b6f94a3-5b5a-40e9-b862-14d9d65cd157-c000.snappy.parquet, Size: 23.04 MB
File: part-00002-4b6f94a3-5b5a-40e9-b862-14d9d65cd157-c000.snappy.parquet, Size: 23.06 MB
File: part-00003-4b6f94a3-5b5a-40e9-b862-14d9d65cd157-c000.snappy.parquet, Size: 23.05 MB


In [32]:

# Calculate the total size of all Parquet files
total_size_bytes = 0
for file in parquet_files:
    file_path = os.path.join(parquet_dir, file)
    total_size_bytes += os.path.getsize(file_path)

# Calculate the average size in MB
average_size_MB = total_size_bytes / len(parquet_files) / (1024 * 1024)

print(f"Average size of the .parquet files: {average_size_MB} MB")

Average size of the .parquet files: 23.042235136032104 MB


## Question 3

Answer - 125,567

"We accidentally discovered that for Q3 homework 5 different versions of Spark give different answers"

How many taxi trips were there on the 15th of October?

Consider only trips that started on the 15th of October.

In [33]:
# Print the schema of the DataFrame (structure of the columns)
df.printSchema()

# Get the column names (headers)
print(df.columns)


root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)

['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fw

In [35]:
from pyspark.sql.functions import col, to_timestamp

# Convert the 'lpep_pickup_datetime' to timestamp if it is not already
df = df.withColumn("lpep_pickup_datetime", to_timestamp("tpep_pickup_datetime", "yyyy-MM-dd HH:mm:ss"))

# Filter the rows for trips on the 15th of October
oct_15_trips = df.filter(col("lpep_pickup_datetime").between("2024-10-15 00:00:00", "2024-10-15 23:59:59"))

# Count the number of trips on the 15th of October
oct_15_count = oct_15_trips.count()

print(f"Number of taxi trips on the 15th of October: {oct_15_count}")


Number of taxi trips on the 15th of October: 128893


## Question 4

Answer - 162

What is the length of the longest trip in the dataset in hours?


In [36]:
from pyspark.sql.functions import col, unix_timestamp

# Calculate the duration in seconds
df_with_duration = df.withColumn(
    "duration_seconds",
    (unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime"))
)

# Convert the duration to hours
df_with_duration = df_with_duration.withColumn("duration_hours", col("duration_seconds") / 3600)

# Find the trip with the longest duration in hours
longest_trip_in_hours = df_with_duration.orderBy(col("duration_hours").desc()).first()

# Print the details of the longest trip
print("Longest trip details (in hours):")
print(longest_trip_in_hours)


Longest trip details (in hours):
Row(VendorID=2, tpep_pickup_datetime=datetime.datetime(2024, 10, 16, 13, 3, 49), tpep_dropoff_datetime=datetime.datetime(2024, 10, 23, 7, 40, 53), passenger_count=1, trip_distance=32.37, RatecodeID=3, store_and_fwd_flag='N', PULocationID=48, DOLocationID=265, payment_type=2, fare_amount=152.5, extra=0.0, mta_tax=0.0, tip_amount=0.0, tolls_amount=17.38, improvement_surcharge=1.0, total_amount=170.88, congestion_surcharge=0.0, Airport_fee=0.0, lpep_pickup_datetime=datetime.datetime(2024, 10, 16, 13, 3, 49), duration_seconds=585424, duration_hours=162.61777777777777)


## Question 5

Answer - 4040

Spark’s User Interface which shows the application's dashboard runs on which local port?


https://spark.apache.org/docs/3.5.4/cluster-overview.html

## Question 6

Answer -  Governor's Island/Ellis Island/Liberty Island



In [37]:
!wget 'https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv'

--2025-03-04 02:37:21--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 52.85.39.65, 52.85.39.97, 52.85.39.153, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|52.85.39.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: ‘taxi_zone_lookup.csv’

taxi_zone_lookup.cs 100%[===================>]  12.04K  --.-KB/s    in 0s      

2025-03-04 02:37:21 (236 MB/s) - ‘taxi_zone_lookup.csv’ saved [12331/12331]



In [38]:
# Load the CSV file into a Spark DataFrame
zone_df = spark.read.csv('taxi_zone_lookup.csv', header=True, inferSchema=True)

# Show the first few rows to verify the data
zone_df.show(5)


+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [39]:
# Create a temporary view
zone_df.createOrReplaceTempView("zone_lookup")

# Verify by running a simple SQL query on the temporary view
spark.sql("SELECT * FROM zone_lookup LIMIT 5").show()


+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+



In [40]:
df.createOrReplaceTempView("yellow_tripdata")


In [41]:
# SQL query to find the least frequent pickup location zone
query = """
    SELECT z.Zone, COUNT(*) as trip_count
    FROM yellow_tripdata y
    JOIN zone_lookup z
    ON y.PULocationID = z.LocationID
    GROUP BY z.Zone
    ORDER BY trip_count ASC
    LIMIT 1
"""

# Execute the SQL query
least_frequent_zone = spark.sql(query)

# Show the result
least_frequent_zone.show()


+--------------------+----------+
|                Zone|trip_count|
+--------------------+----------+
|Governor's Island...|         1|
+--------------------+----------+

